In [28]:
import pandas as pd
import numpy as np
from my_tools import get_bill_data, process_corpus
import matplotlib.pyplot as plt

In [2]:
data = get_bill_data()

data.describe()

------------------
------------------
Data is that with text from the 110th Congress (2007) to present
Alter masking in my_tools.get_bill_data to get a different data set.
------------------


In [33]:
data.head()

,_id,bill_status,body,committee,congress_id,cosponsors,cosponsors_url,desc,intro_date,leg_id,...,num_of_cosponsors,sponsor,sponsor_district,sponsor_party,sponsor_state,bill_char_counts,intro_month,session,char_count_bucket,labels
0,5c182d561417de23a8260f58,Became Law,[110th Congress Public Law 180] [From the U.S....,House - Judiciary | Senate - Judiciary,110th,None,https://www.congress.gov/bill/110th-congress/h...,NICS Improvement Amendments Act of 2007,2007-06-11,H R 2640,...,17,"Rep. McCarthy, Carolyn",4,D,NY,432,6,2,less than 1000,1
1,5c182d561417de23a8261074,Became Law,[110th Congress Public Law 239] [From the U.S....,House - Judiciary | Senate - Judiciary,110th,None,https://www.congress.gov/bill/110th-congress/h...,"To amend title 4, United States Code, to encou...",2007-05-17,H R 2356,...,1,"Rep. Scott, David",13,D,GA,924,5,2,less than 1000,1
2,5c182d571417de23a82619aa,Became Law,[110th Congress Public Law 137] [From the U.S....,House - Appropriations,110th,None,None,Making further continuing appropriations for t...,2007-12-12,H J Res 69,...,0,"Rep. Obey, David R.",7,D,WI,797,12,2,less than 1000,1
3,5c182d581417de23a82621c3,Became Law,[110th Congress Public Law 80] [From the U.S. ...,"Senate - Agriculture, Nutrition, and Forestry",110th,None,https://www.congress.gov/bill/110th-congress/s...,"A bill to amend the U.S. Troop Readiness, Vete...",2007-06-27,S 1716,...,12,"Sen. Thune, John",None,R,SD,967,6,2,less than 1000,1
4,5c182d581417de23a8262651,Became Law,[110th Congress Public Law 201] [From the U.S....,Senate - Homeland Security and Governmental Af...,110th,None,https://www.congress.gov/bill/110th-congress/s...,A bill to preserve existing judgeships on the ...,2007-02-12,S 550,...,2,"Sen. Akaka, Daniel K.",None,D,HI,884,2,2,less than 1000,1


In [45]:
#  pass corpus through nlp pipeline
X, y = process_corpus(data, 'body')

------------------
Creating corpus...
------------------
Tokenizing...
------------------
Stripping out stop words, punctuation, and numbers...
------------------
Lemmatizing...
------------------
Creating a vocabulary...
------------------
NLP preprocessing complete ...


In [39]:
data.describe()

,num_of_cosponsors,bill_char_counts,intro_month,session
count,52238.000000,5.223800e+04,52238.000000,52238.000000
mean,12.243099,1.598837e+04,5.518243,1.613634
std,29.163586,7.222059e+04,3.265099,0.486921
min,0.000000,4.320000e+02,1.000000,1.000000
25%,0.000000,2.184000e+03,3.000000,1.000000
50%,2.000000,4.497000e+03,5.000000,2.000000
75%,11.000000,1.099750e+04,8.000000,2.000000
max,432.000000,2.758251e+06,12.000000,2.000000


In [41]:
len(data[data['num_of_cosponsors'] > 200])

270

In [34]:
data.columns

Index(['_id', 'bill_status', 'body', 'committee', 'congress_id', 'cosponsors',
       'cosponsors_url', 'desc', 'intro_date', 'leg_id', 'leg_type', 'leg_url',
       'num_of_cosponsors', 'sponsor', 'sponsor_district', 'sponsor_party',
       'sponsor_state', 'bill_char_counts', 'intro_month', 'session',
       'char_count_bucket', 'labels'],
      dtype='object')

In [35]:
data.sponsor_party.value_counts()

D     30434
R     21437
I       224
ID      142
Name: sponsor_party, dtype: int64

In [36]:
data.labels.value_counts()

0    50158
1     2080
Name: labels, dtype: int64

In [ ]:
columns = ['sponsor', 'num_of_bills']
sponsor_counts = pd.DataFrame(data.sponsor.value_counts()).reset_index()
sponsor_counts.columns = columns

In [27]:
len(sponsor_counts)

966

In [29]:
x = np.arange(len(sponsor_counts))

In [32]:
sponsor_counts.describe()

,num_of_bills
count,966.000000
mean,54.075569
std,59.082602
min,1.000000
25%,17.000000
50%,37.000000
75%,66.000000
max,481.000000


In [42]:
sponsor_counts.head()

,sponsor,num_of_bills
0,"Sen. Schumer, Charles E.",481
1,"Sen. Menendez, Robert",471
2,"Sen. Feinstein, Dianne",456
3,"Sen. Casey, Robert P., Jr.",448
4,"Sen. Vitter, David",425
